<a href="https://colab.research.google.com/github/nhwagle/code-match/blob/main/Equation_Focused_Primary_Method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports

Installing codegen. Their install_env.sh won't work for colab so modifying such that all conda imports are done with pip

In [ ]:
%%capture
%pip install transformers pylatexenc

In [ ]:
import torch
import torch.nn as nn
import transformers
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random


from transformers import DistilBertTokenizer, DistilBertModel
from pylatexenc.latex2text import LatexNodes2Text

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ls /content/gdrive/MyDrive/RSS/Data

algorithms.csv	code.csv	  code-small.csv  func-rep-small.csv
code-all.csv	code-func-rep.pt  equations.csv   func-rep-small.pt


In [ ]:
eqns = pd.read_csv("/content/gdrive/MyDrive/RSS/Data/equations.csv")
algs = pd.read_csv("/content/gdrive/MyDrive/RSS/Data/algorithms.csv")
code = pd.read_csv("/content/gdrive/MyDrive/RSS/Data/code.csv")

Comparing methods for reading in latex

In [ ]:

model_class = transformers.DistilBertModel 
tokenizer_class = transformers.DistilBertTokenizer
weights = 'distilbert-base-cased'

tokenizer = tokenizer_class.from_pretrained(weights)
model = model_class.from_pretrained(weights)


Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/251M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
eqn_index = random.randint(0,10000)
#bellman:
# eqn_index = 3210


print(f"Example of equation index number {eqn_index}")

latex = eqns.iloc[eqn_index].eqn
print("")
print("======")
print("Multistep")
print("======")
print("Raw latex below:")
print("")
print(latex)
print("Converted Unicode below:")
uni_rep = LatexNodes2Text().latex_to_text(latex)
print(uni_rep)
print("Tokenization below:")
print("")
print(tokenizer(uni_rep,padding="max_length",truncation=True))
print("")
print("Inverted Tokenization")
print("")
print(tokenizer.decode(tokenizer(uni_rep)['input_ids']))

print("======")
print("Single step")
print("======")
print("Inverted Tokenization")
print("")
print(tokenizer.decode(tokenizer(latex)['input_ids']))

Example of equation index number 2311

Multistep
Raw latex below:

\label{eq:alifZ}
            z^t_j = H\left(v_j^t - v_\text{th} - \beta a^t_j\right)
            
Converted Unicode below:

            z^t_j = H(v_j^t - v_th - βa^t_j)
            
Tokenization below:

{'input_ids': [101, 195, 167, 189, 168, 179, 134, 145, 113, 191, 168, 179, 167, 189, 118, 191, 168, 24438, 118, 419, 1161, 167, 189, 168, 179, 114, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

Appears to be how to work with tokenized latex...

In [ ]:
model(**tokenizer(latex,return_tensors='pt',padding='max_length',truncation=True))[0].shape

torch.Size([1, 512, 768])

In [ ]:
test = tokenizer(latex,return_tensors='pt',padding='max_length',truncation=True)
test

{'input_ids': tensor([[  101,   165,  3107,   196,   174,  4426,   131,  2393,  8914,  5301,
           198,   195,   167,   189,   168,   179,   134,   145,   165,  1286,
           113,   191,   168,   179,   167,   189,   118,   191,   168,   165,
          3087,   196, 24438,   198,   118,   165, 11933,   170,   167,   189,
           168,   179,   165,  1268,   114,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

In [ ]:
!ls gdrive/MyDrive/RSS/Data

algorithms.csv	code.csv	  code-small.csv  func-rep-small.csv
code-all.csv	code-func-rep.pt  equations.csv   func-rep-small.pt


In [ ]:
func_rep_df = pd.read_pickle('gdrive/MyDrive/RSS/Data/code-func-rep.pt')

In [ ]:
func_rep_df = func_rep_df[func_rep_df.functions.apply(len) != 0]
func_rep_df

,id,functions,representations
0,1811.02182,[def main ( config ) : NEW_LINE INDENT from da...,"[[[tensor(0.6232), tensor(-0.2805), tensor(0.3..."
1,1811.02182,"[def check_config_used ( config , target_sourc...","[[[tensor(0.6108), tensor(-0.2560), tensor(0.1..."
2,1811.02182,"[def __init__ ( self , batch_size , paired = F...","[[[tensor(0.8430), tensor(0.1034), tensor(0.30..."
3,1811.02182,[def weights_init ( m ) : NEW_LINE INDENT clas...,"[[[tensor(0.6790), tensor(-0.0221), tensor(0.1..."
4,1811.02182,[def weights_init ( m ) : NEW_LINE INDENT clas...,"[[[tensor(0.6790), tensor(-0.0221), tensor(0.1..."
...,...,...,...
29193,2112.01073,"[def my_lcs ( string , sub ) : NEW_LINE INDENT...","[[[tensor(0.6292), tensor(0.0059), tensor(0.09..."
29194,2112.01073,"[def __init__ ( self , n = 4 ) : NEW_LINE INDE...","[[[tensor(0.4932), tensor(-0.1197), tensor(-0...."
29195,2112.01073,"[def precook ( s , n = 4 , out = False ) : NEW...","[[[tensor(0.7558), tensor(-0.0280), tensor(0.0..."
29197,2112.01073,[def __init__ ( self ) : NEW_LINE INDENT self ...,"[[[tensor(0.7251), tensor(-0.0360), tensor(0.0..."


In [ ]:
eqns['id'] = eqns.id.astype(str)
func_rep_df['id'] = func_rep_df['id'].astype(str)
collected_func_reps = func_rep_df.groupby(by='id').sum()

In [ ]:
demofuncs = collected_func_reps[collected_func_reps.index=='1811.02182']
demofuncs.functions

id
1811.02182    [def main ( config ) : NEW_LINE INDENT from da...
Name: functions, dtype: object

In [ ]:
demofuncs.functions[0]

["def main ( config ) : NEW_LINE INDENT from data_loader import DataLoader NEW_LINE if ( config . trainer == ' minimize _ DCE ' ) : NEW_LINE INDENT from trainer_DCE import Trainer NEW_LINE paired = True NEW_LINE DEDENT elif ( config . trainer == ' acoustic _ supervision ' ) : NEW_LINE INDENT from trainer_acoustic import Trainer NEW_LINE paired = False NEW_LINE DEDENT elif ( config . trainer == ' AAS ' ) : NEW_LINE INDENT from trainer_AAS import Trainer NEW_LINE paired = False NEW_LINE DEDENT elif ( config . trainer == ' FSEGAN ' ) : NEW_LINE INDENT from trainer_FSEGAN import Trainer NEW_LINE paired = True NEW_LINE DEDENT if config . gpu >= 0 : NEW_LINE INDENT torch . cuda . manual_seed ( config . random_seed ) NEW_LINE torch . cuda . set_device ( config . gpu ) NEW_LINE DEDENT if ( config . DB_name == ' librispeech ' ) : NEW_LINE INDENT if ( paired ) : NEW_LINE INDENT config . tr_ny_manifest = ' data / libri _ tr _ ny _ paired . csv ' NEW_LINE config . trsub_manifest = ' data / libri _

In [ ]:
full_df = eqns.merge(collected_func_reps , on='id')

In [ ]:
for row in full_df.iterrows():
  print(tokenizer(row[1].eqn,return_tensors='pt',truncation=True))
  break

{'input_ids': tensor([[  101,   165,  3295,   196,  3325,   198,   165, 11241,   168,   196,
           144,   198,   165, 12477,  1775,   168,   196,   141,   198,   159,
           168,   196,   172, 10583,  2249,   198,   113,   144,   117,   141,
           114,   134,   165,   142,   168,   196,   113,   165,  3087,  1830,
          2087,   196,   193,   198,   168,   196,   188,   198,   117,   165,
          3087,  1830,  2087,   196,   193,   198,   168,   196,   189,   198,
           114,   165, 27466,  1306,   185,   113,   165,  3087,  1830,  2087,
           196,   193,   198,   168,   196,   188,   198,   117,   165,  3087,
          1830,  2087,   196,   193,   198,   168,   196,   189,   198,   114,
           198,   164,   165,  9366,   141,   113,   165,  3087,  1830,  2087,
           196,   193,   198,   168,   188,   117,   165,  3087,  1830,  2087,
           196,   193,   198,   168,   196,   189,   198,   114,   166,   165,
           165,   116,   165,   142,  

In [ ]:
class ContrastiveDataset(Dataset):

    def __init__(self, combined_df, all_eqns):
        """Simple init function"""
        self.df = combined_df
        tok_eqns = []
        for row in self.df.iterrows():
          tok_eqns.append(tokenizer(row[1].eqn,return_tensors='pt',truncation=True))
        self.tok_eqns = tok_eqns

        self.neg_eqns = all_eqns
        neg_eqns = []
        for item in all_eqns:
          neg_eqns.append(tokenizer(item,return_tensors='pt',truncation=True))

        self.neg_toks = neg_eqns


    def __len__(self):
        """Get length of dataset"""
        return len(self.df)


    def __getitem__(self, idx):
        entry = self.df.iloc[idx]

        eqn_paper_id = entry.id
        code_id = eqn_paper_id 
        code_rep = entry.representations
        code_funcs = entry.functions

        pos_eqn = entry.eqn
        pos_eqn_tok = self.tok_eqns[idx]

        neg_idx = random.randint(0,len(self.neg_eqns)-1)
        neg_eqn = self.neg_eqns.iloc[neg_idx]
        neg_eqn_tok = self.neg_toks[neg_idx]

        return code_rep, pos_eqn_tok, neg_eqn_tok, pos_eqn, neg_eqn, eqn_paper_id, code_id, code_funcs

        


In [ ]:
train_dataset = ContrastiveDataset(full_df, eqns.eqn.dropna())

In [ ]:
class ContrastiveModel(nn.Module):
  def __init__(self, embedding_model):
    super(ContrastiveModel,self).__init__()

    self.latex_encoder = embedding_model

    self.remap_eqn_fc1 = nn.Linear(768,768)
    # self.reduce_eqn_fc2 = nn.Linear(768,1024)
    self.expand_eqn_fc3 = nn.Linear(768,1024)

    self.remix_res_fc1 = nn.Linear(1024,1024)
    self.remix_res_fc2  = nn.Linear(1024,768)

    self.softmax = nn.Softmax(dim=0)
    self.eqn_drop = nn.Dropout(p=0.5)
    


  def embed_eqn(self, eqn):
      return self.latex_encoder(input_ids=eqn[0],attention_mask=eqn[1])[0].mean(-2)

  def reduce_eqn(self,eqn_embedding):
      eqn_embedding = self.eqn_drop(eqn_embedding)
      h = self.remap_eqn_fc1(eqn_embedding)
      h = F.relu(h)
      h = self.expand_eqn_fc3(h)
      h = F.normalize(h)
      return h


  def forward(self, func_reps,pos_eqn, neg_eqn):
      # get embeddings
      pos_eqn_embedding = self.embed_eqn(pos_eqn)
      neg_eqn_embedding = self.embed_eqn(neg_eqn)


      #reduce the embedding size of pos_h and then expand it to match func_reps
      #the expansion is concerning, but the hope is we reduce the information           
      pos_h_norm = self.reduce_eqn(pos_eqn_embedding)

      #norm func reps as well because we want the next operation to correspond 
      #to cosine similarity
      normed_reps = F.normalize(func_reps,dim=-1)
      # print(normed_reps)
     
      #get an attention map
      attn_map = torch.matmul(normed_reps.squeeze(1),pos_h_norm.squeeze())

      #zero out so don't have negatives influencing results
      pre_soft_attn_map = F.relu(attn_map)
      #convert to probabilities
      attn_map = self.softmax(pre_soft_attn_map)

      #recombine function reps based on attention
      z = torch.matmul(attn_map,func_reps.squeeze()) if func_reps.shape[0] > 1 else func_reps

      z = F.relu(z)
      z = self.remix_res_fc1(z)
      z = F.relu(z)
      z = self.remix_res_fc2(z)

      return z, pos_eqn_embedding, neg_eqn_embedding, attn_map



In [ ]:
train_loader = DataLoader(train_dataset,batch_size=1,shuffle=True)

In [ ]:
model = DistilBertModel.from_pretrained("distilbert-base-cased")

combined_model = ContrastiveModel(model)
combined_model.train()
combined_model.cuda()

optimizer = optim.Adam(combined_model.parameters(), lr=5e-5)
criterion = nn.TripletMarginLoss()




Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
!nvidia-smi

Wed Apr 27 00:53:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    33W / 250W |   4735MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
epochs = 30

In [ ]:
combined_model.train()
accum_iter = 256
for epoch in range(epochs):
    print("--------")
    print(f"Epoch {epoch}")
    print("--------")
    correct = 0
    total = 0
    loss_avg = 0
    for batch_idx,input in enumerate(train_loader):

     

      code_rep, pos_eqn_tok, neg_eqn_tok, pos_eqn, neg_eqn, eqn_paper_id, code_id, code_funcs = input  
      # if len(code_funcs) < 1:
      #     print('continuing')
      #     continue    



      # forward + backward + optimize
      curr_rep = torch.cat(code_rep).cuda()
      pos_eqn_prep = pos_eqn_tok['input_ids'].flatten(1).cuda(),pos_eqn_tok['attention_mask'].flatten(1).cuda()
      neg_eqn_prep = neg_eqn_tok['input_ids'].flatten(1).cuda(),neg_eqn_tok['attention_mask'].flatten(1).cuda()

      # output
      z,pos,neg,attn = combined_model(curr_rep,pos_eqn_prep,neg_eqn_prep)
      
      loss = criterion(z.squeeze(),pos.squeeze(),neg.squeeze()) 
      
      loss = loss / accum_iter
      loss.backward()

      if ((batch_idx + 1) % accum_iter == 0) or (batch_idx + 1 == len(train_loader)):
            optimizer.step()
            optimizer.zero_grad()
            
            print(f"{loss_avg}")
            
            loss_avg = 0
            print(f"On batch {batch_idx}")

            print(attn)
            print(pos_eqn)
            print(code_funcs)


      loss_avg += loss.item()
    


--------
Epoch 0
--------
2.7579037696123123
On batch 255
tensor([0.0010, 0.0010, 0.0010,  ..., 0.0010, 0.0010, 0.0010], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)
('\\label{IoU}\n% \\mathit{IoU} = \\frac{area(B_r\\cap B_{gt})}{area(B_r\\cup B_{gt})}\n% ',)
[("def get_segmentation_train_batch ( segdb , config ) : NEW_LINE INDENT assert len ( segdb ) == 1 , ' Single ▁ batch ▁ only ' NEW_LINE imgs , seg_cls_gts , segdb = get_segmentation_image ( segdb , config ) NEW_LINE im_array = imgs [ 0 ] NEW_LINE seg_cls_gt = seg_cls_gts [ 0 ] NEW_LINE im_info = np . array ( [ segdb [ 0 ] [ ' im _ info ' ] ] , dtype = np . float32 ) NEW_LINE data = { ' data ' : im_array , ' im _ info ' : im_info } NEW_LINE label = { ' label ' : seg_cls_gt } NEW_LINE return data , label NEW_LINE DEDENT",), ('def show_boxes ( im , dets , classes , scale = 1.0 ) : NEW_LINE INDENT plt . cla ( ) NEW_LINE plt . axis ( " off " ) NEW_LINE plt . imshow ( im ) NEW_LINE for cls_idx , cls_name in enumerate ( classes ) 

KeyboardInterrupt: ignored

In [ ]:
code_rep

[tensor([[[ 0.7814, -0.1531,  0.2438,  ...,  0.5276,  0.1782,  0.2289]]]),
 tensor([[[ 0.4640, -0.0946,  0.2987,  ...,  0.4522,  0.2744,  0.2990]]]),
 tensor([[[ 0.9164, -0.3907,  0.0960,  ...,  0.5478,  0.3017, -0.0221]]]),
 tensor([[[ 0.9199, -0.3740,  0.1033,  ...,  0.5090,  0.2973, -0.0067]]]),
 tensor([[[ 0.9440, -0.3906,  0.1073,  ...,  0.5119,  0.3007, -0.0176]]]),
 tensor([[[ 0.8080, -0.2522,  0.1362,  ...,  0.4910,  0.2463,  0.0997]]]),
 tensor([[[ 0.7899, -0.2610,  0.1008,  ...,  0.4811,  0.2407,  0.0743]]]),
 tensor([[[ 0.8414, -0.2704,  0.1358,  ...,  0.4767,  0.2585,  0.0918]]]),
 tensor([[[ 0.7523, -0.2260,  0.0746,  ...,  0.4840,  0.2679,  0.1212]]]),
 tensor([[[ 0.8664, -0.1233,  0.0296,  ...,  0.3527,  0.2741,  0.1871]]]),
 tensor([[[ 0.7946, -0.1043,  0.0557,  ...,  0.3444,  0.2854,  0.1643]]]),
 tensor([[[ 0.8321, -0.1260,  0.0382,  ...,  0.3618,  0.2802,  0.1615]]]),
 tensor([[[ 0.8621, -0.1249,  0.0124,  ...,  0.3173,  0.2748,  0.2579]]]),
 tensor([[[ 0.7980, -0.04

In [ ]:
import time
torch.save(combined_model.state_dict(),"gdrive/MyDrive/RSS/Models/combined_model"+str(time.strftime('%Y-%m-%d_%H:%M:%S.pt', time.localtime())) )